# Multiple answer cleanup experiment
Attempt at dealing with fields that contain comma-separated lists of answers from questions with multiple-select answer options that also allow respondents to enter their own 'other' answer.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

## Practice data set
Simulating how responses to multiple-select answers options are stored by Google Forms

In [2]:
df = pd.DataFrame({
    'a': ['foo, nonsense1', 'nonsense2, bar'],
    'b': ['nonsense3, hello', 'world, nonsense4'],
    'c': ['nonesuch, forthwith', 'nonsense5, forthwith']
})
df

,a,b,c
0,"foo, nonsense1","nonsense3, hello","nonesuch, forthwith"
1,"nonsense2, bar","world, nonsense4","nonsense5, forthwith"


## Practice answer options to each question
A dictionary containing lists of all the answer options available to each question in the Google Form.

In [3]:
# a dictionary of columns that contain comma-separated multiple answer options
multiple_answer_options = {
    'a': ['foo', 'bar'],
    'b': ['hello', 'world'],
    'c': ['nonesuch', 'forthwith'],
    # other option available for some
}


## Remove commas from any given answer option so we can later split by comma cleanly

In [4]:
# loop through all questions that allow more than one answer
for question, answer_list in multiple_answer_options.items():
    # loop through every answer in the answer list for this question
    for answer in answer_list:
        # remove the commas, from each answer, if any
        if ',' in answer:
            # update the answers with a cleaned version
            df[question] = df[question].str.replace(answer, answer.replace(',', '') )
            
    # remove commas from the list of answer in the dictionary
    multiple_answer_options[question] = [answer.replace(',' , '') for answer in answer_list]
    #display(multiple_answer_options[question])

# show the cleaned up answer options
multiple_answer_options

{'a': ['foo', 'bar'], 'b': ['hello', 'world'], 'c': ['nonesuch', 'forthwith']}

## First try at cleaning up answers
Currently abandoned.

In [5]:
def add_multiple_answer_columns(df, question, answer_list):
    for answer in answer_list:
        ... what to do here??!

def categorize_answers(response, question, answer_list):
    # convert to a series with the answer as a key
    indices = [] # will contain the indices for the series of answers
    values = [] # will contain the values for the series of answers
    
    # skip any nan or blank values
    if str(response) != 'nan' and type(answer_list) != float and type(question) != float and type(response) != float:
        #display(question + " - " + str(response))
        for answer_option in response:
            answer_option = answer_option.strip() # remove any leading/trailing whitespace
            if answer_option in answer_list:
                indices.append(answer_option)
                values.append(1)
            else:
                indices.append('other')
                values.append(answer_option)
    
        response = pd.Series(values,index=indices).fillna(0).astype(str)
        #display(response)
        
    return response

# slice up answers by comma and give each its own column
df2 = pd.DataFrame()
for index, row in df.iterrows():
    # process each response from the user to this question
    #d = row[question].str.split(',\s*').apply(categorize_answers, question=question, answer_list=answer_list)
    display(row)
    #df2 = df2.append(d, sort=False)

#df2

SyntaxError: invalid syntax (<ipython-input-5-d8acd4aca7e3>, line 3)

## Second try
Counts how many times each answer has been given to each question, and groups freeform text answers into an 'other' column.

In [13]:
def categorize_answers(response, question, answer_list):
    # convert to a series with the answer as a key
    indices = [] # will contain the indices for the series of answers
    values = [] # will contain the values for the series of answers
    
    # skip any nan or blank values
    if str(response) != 'nan' and type(answer_list) != float and type(question) != float and type(response) != float:
        #display(question + " - " + str(response))
        for answer_option in response:
            answer_option = answer_option.strip() # remove any leading/trailing whitespace
            if answer_option in answer_list:
                indices.append(answer_option)
                values.append(1)
            else:
                indices.append('other')
                values.append(answer_option)
    
        response = pd.Series(values,index=indices).fillna(0).astype(str)
        #display(response)
        
    return response

# slice up answers by comma and give each its own column
df2 = pd.DataFrame()
for question, answer_list in multiple_answer_options.items():
    # process each response from the user to this question
    d = df[question].str.split(',\s*').apply(categorize_answers, question=question, answer_list=answer_list)
    #display(d)
    df2 = df2.append(d, sort=False)

# replace NaN with zeros ... don't do this if counting the number of responses in each column
#df2.fillna(0, inplace=True)

# show the results
df2

,foo,other,bar,hello,world,nonesuch,forthwith
0,1,nonsense1,NaN,NaN,NaN,NaN,NaN
1,NaN,nonsense2,1,NaN,NaN,NaN,NaN
0,NaN,nonsense3,NaN,1,NaN,NaN,NaN
1,NaN,nonsense4,NaN,NaN,1,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,1,1
1,NaN,nonsense5,NaN,NaN,NaN,NaN,1


In [12]:
sum_column = df2.count(axis=0)
print(sum_column)

foo          1
other        5
bar          1
hello        1
world        1
nonesuch     1
forthwith    2
dtype: int64
